# **Procesamiento Distribuido con PySpark en Pipelines en Azure ML**


## **Módulo 1: Introducción a PySpark (1 hora)**  
**Objetivo:** Comprender qué es PySpark, cómo funciona y cuándo usarlo en comparación con herramientas como Pandas.

---

### **1. ¿Qué es PySpark y cómo funciona?**
- **Definición:**  
  PySpark es la API de Python para Apache Spark, un motor de procesamiento distribuido diseñado para manejar grandes volúmenes de datos de manera eficiente.
- **Funcionamiento:**  
  - Utiliza un modelo de computación distribuida basado en clústeres.
  - Divide los datos en particiones y procesa cada partición en paralelo en múltiples nodos.
  - Proporciona abstracciones como RDDs (Resilient Distributed Datasets) y DataFrames para manipular datos.

#### **1.1. Comparación con Pandas**
| **Característica**       | **Pandas**                          | **PySpark**                         |
|--------------------------|-------------------------------------|-------------------------------------|
| **Escalabilidad**        | Limitado a datos en memoria local. | Escalable a grandes volúmenes de datos distribuidos. |
| **Velocidad**            | Lento para grandes conjuntos de datos. | Rápido gracias al procesamiento paralelo. |
| **Uso Recomendado**      | Datos pequeños o medianos.         | Grandes volúmenes de datos (Big Data). |

- **Cuándo usar PySpark:**  
  - Cuando el volumen de datos excede la capacidad de memoria local.
  - Cuando necesitas procesar datos en tiempo real o realizar análisis distribuidos.
  - Para integrar pipelines de machine learning con Big Data.

#### **Referencias:**
- [Apache Spark Documentation](https://spark.apache.org/docs/latest/)
- [PySpark vs Pandas Comparison](https://spark.apache.org/docs/latest/api/python/)



## **Módulo 2: Configuración de PySpark en Azure (1 hora)**  
**Objetivo:** Aprender a configurar PySpark en Azure utilizando Azure Databricks y conectarlo con Azure Blob Storage.

---

### **2.1. Uso de Azure Databricks para Ejecutar PySpark**
Azure Databricks es un servicio administrado que facilita la ejecución de trabajos de PySpark en clústeres escalables.

- **Pasos para Configurar Azure Databricks:**
  1. Crea un área de trabajo de Databricks en Azure Portal.
  2. Configura un clúster de Spark en Databricks:
     - Selecciona el número de nodos y el tipo de máquina virtual.
     - Instala bibliotecas adicionales si es necesario (e.g., `azure-storage-blob`).
  3. Conecta Databricks con Azure Machine Learning Workspace para integrarlo en pipelines.

#### **Ejemplo Práctico: Creación de un Clúster en Databricks**
```python
from pyspark.sql import SparkSession

# Inicializar SparkSession
spark = SparkSession.builder \
    .appName("ProcesamientoDistribuido") \
    .getOrCreate()

print("Clúster de Spark inicializado.")
```

---

### **2.2. Conexión con Azure Blob Storage**
Azure Blob Storage es una solución ideal para almacenar grandes volúmenes de datos que se pueden procesar con PySpark.

- **Pasos para Conectar Blob Storage:**
  1. Configura una cuenta de almacenamiento en Azure Portal.
  2. Usa la biblioteca `azure-storage-blob` para cargar/descargar datos.
  3. Monta el Blob Storage en Databricks para acceso directo.

#### **Ejemplo Práctico: Lectura de Datos desde Blob Storage**
```python
# Montar Blob Storage en Databricks
dbutils.fs.mount(
    source="wasbs://<container-name>@<storage-account>.blob.core.windows.net",
    mount_point="/mnt/mi-datos",
    extra_configs={"fs.azure.account.key.<storage-account>.blob.core.windows.net": "<storage-key>"}
)

# Leer datos desde Blob Storage
df = spark.read.csv("/mnt/mi-datos/dataset.csv", header=True, inferSchema=True)
df.show()
```

#### **Referencias:**
- [Azure Databricks Documentation](https://learn.microsoft.com/en-us/azure/databricks/)
- [Azure Blob Storage Integration](https://learn.microsoft.com/en-us/azure/storage/blobs/)



## **Módulo 3: Operaciones Básicas con PySpark (2 horas)**  
**Objetivo:** Aprender a cargar, transformar y analizar datos utilizando PySpark.

---

### **3.1. Carga y Transformación de Datos**
- **Carga de Datos:**  
  PySpark soporta múltiples formatos como CSV, JSON, Parquet y más.
  ```python
  # Cargar un archivo CSV
  df = spark.read.csv("datos.csv", header=True, inferSchema=True)
  ```

- **Transformaciones Comunes:**  
  - Selección de columnas:
    ```python
    df.select("columna1", "columna2").show()
    ```
  - Filtrado de datos:
    ```python
    df.filter(df["edad"] > 30).show()
    ```
  - Renombrar columnas:
    ```python
    df.withColumnRenamed("old_name", "new_name").show()
    ```

---

### **3.2. Agregaciones y Join de Tablas Distribuidas**
- **Agregaciones:**  
  Realiza operaciones como sumas, promedios y conteos.
  ```python
  # Calcular el promedio de una columna
  df.groupBy("categoria").agg({"valor": "avg"}).show()
  ```

- **Join de Tablas:**  
  Combina múltiples tablas distribuidas.
  ```python
  # Unir dos DataFrames
  df1.join(df2, df1.id == df2.id, "inner").show()
  ```

#### **Referencias:**
- [PySpark DataFrame Operations](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html)



## **Módulo 4: Integración con Pipelines (2 horas)**  
**Objetivo:** Aprender a integrar PySpark en pipelines de Azure ML para preprocesar datos antes de entrenar modelos.

---

### **4.1. Preprocesamiento de Datos con PySpark**
PySpark es ideal para preprocesar grandes volúmenes de datos antes de entrenar modelos en Azure ML.

- **Pasos Comunes:**
  1. Limpiar y normalizar los datos.
  2. Realizar ingeniería de características.
  3. Exportar los datos procesados para entrenamiento.

#### **Ejemplo Práctico: Preprocesamiento con PySpark**
```python
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Ingeniería de características
assembler = VectorAssembler(inputCols=["feature1", "feature2"], outputCol="features")
df_transformed = assembler.transform(df)

# Normalización
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(df_transformed)
df_scaled = scaler_model.transform(df_transformed)

df_scaled.show()
```

---

### **4.2. Ejemplo Práctico: Procesamiento de Datos Masivos para un Modelo de Clustering**
Supongamos que queremos entrenar un modelo de clustering en Azure ML utilizando datos procesados con PySpark.

#### **Pasos:**
1. **Procesamiento con PySpark:**
   - Limpia y transforma los datos.
   - Exporta los datos procesados a Azure Blob Storage.
2. **Entrenamiento del Modelo:**
   - Carga los datos procesados en Azure ML.
   - Entrena un modelo de clustering (e.g., K-Means).

#### **Ejemplo Práctico: Pipeline Completo**
```python
# Paso 1: Procesamiento con PySpark
df.write.format("parquet").save("/mnt/mi-datos/procesados.parquet")

# Paso 2: Entrenamiento en Azure ML
from azureml.core import Dataset
from sklearn.cluster import KMeans

# Cargar datos procesados
dataset = Dataset.Tabular.from_parquet_files(path=(datastore, 'procesados.parquet'))
X = dataset.to_pandas_dataframe()

# Entrenar modelo
model = KMeans(n_clusters=3)
model.fit(X)
```



## **Actividad Práctica (2 horas)**

**Título:** Procesar un conjunto de datos grande con PySpark y entrenar un modelo de clustering en un pipeline de Azure ML.

#### **Pasos:**
1. **Configuración del Entorno:**
   - Crea un clúster de Databricks en Azure.
   - Monta un contenedor de Blob Storage en Databricks.

2. **Procesamiento con PySpark:**
   - Carga un conjunto de datos grande desde Blob Storage.
   - Realiza limpieza, normalización y agregaciones con PySpark.
   - Guarda los datos procesados en Blob Storage.

3. **Entrenamiento del Modelo:**
   - Carga los datos procesados en Azure ML.
   - Entrena un modelo de clustering utilizando Scikit-learn o AutoML.

4. **Despliegue del Modelo:**
   - Despliega el modelo como un endpoint REST.
   - Prueba el endpoint con datos de ejemplo.

---

### **Conclusión**
Este curso proporciona una visión completa del uso de PySpark para procesamiento distribuido dentro de pipelines de Azure ML. Los participantes estarán preparados para manejar grandes volúmenes de datos y construir pipelines escalables para entrenar modelos de machine learning.

#### **Referencias Generales:**
- [Azure Databricks Documentation](https://learn.microsoft.com/en-us/azure/databricks/)
- [PySpark Documentation](https://spark.apache.org/docs/latest/api/python/)
- [Azure Machine Learning Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/)